In [1]:
import sentence_transformers
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from torch.utils.data import Dataset
from typing import List
from sentence_transformers.readers import InputExample
import numpy as np
from transformers.utils.import_utils import is_nltk_available, NLTK_IMPORT_ERROR
from transformers import AutoTokenizer
from nltk import word_tokenize, TreebankWordDetokenizer
import nltk
nltk.download('punkt')


/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/cgrdj/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:

model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False,clean_up_tokenization_spaces=False,clean_text=False)




config.json: 100%|██████████| 409/409 [00:00<00:00, 1.30MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 985kB/s]


In [3]:

class DenoisingAutoEncoderDataset(Dataset):
    """
    The DenoisingAutoEncoderDataset returns InputExamples in the format: texts=[noise_fn(sentence), sentence]
    It is used in combination with the DenoisingAutoEncoderLoss: Here, a decoder tries to re-construct the
    sentence without noise.

    :param sentences: A list of sentences
    :param noise_fn: A noise function: Given a string, it returns a string with noise, e.g. deleted words
    """

    def __init__(self, sentences: List[str], noise_fn=lambda s: DenoisingAutoEncoderDataset.delete(s)):
        if not is_nltk_available():
            raise ImportError(NLTK_IMPORT_ERROR.format(self.__class__.__name__))

        self.sentences = sentences
        self.noise_fn = noise_fn

    def __getitem__(self, item):
        sent = self.sentences[item]
        return InputExample(texts=[self.noise_fn(sent), sent])

    def __len__(self):
        return len(self.sentences)

    # Deletion noise.
    @staticmethod
    def noisen(text,tokenizer, MASK_ratio=0.15):
        mask_id=tokenizer.mask_token_id
        words= word_tokenize(text)
        # Apply the masking logic to each word and rejoin the sentence
        splitted_tokens = tokenizer.batch_encode_plus(words,return_attention_mask=False,return_token_type_ids=False,add_special_tokens=False)['input_ids']
        [ mask_id if np.random.rand() < MASK_ratio else tok_id for tok_id in word]
        masked_sentence=' '.join([tokenizer.decode([ mask_id if np.random.rand() < MASK_ratio else tok_id for tok_id in word]).replace(" ",'') for word in splitted_tokens])
        return masked_sentence

In [4]:
sentence="asdasdf adsf sda fads 'asd gasdg aasdfasdf /dsaf sadfsd,dasfadsf. fadsf.fads,fadsfa"
tokenizer.decode(tokenizer.encode_plus(sentence,return_attention_mask=False,return_token_type_ids=False,add_special_tokens=False)['input_ids'])



"asdasdf adsf sda fads ' asd gasdg aasdfasdf / dsaf sadfsd, dasfadsf. fadsf. fads, fadsfa"

In [5]:
tokenizer.decode(tokenizer.encode('as [MASK]df',add_special_tokens=False))



'as [MASK] df'

In [6]:

sentences=["asdasdf adsf sda fads 'asd gasdg aasdfasdf /dsaf sadfsd,dasfadsf. fadsf.fads,fadsfa"]*10000
masked_sentences=[]
for sentence in sentences:
    words= word_tokenize(sentence)
    # Apply the masking logic to each word and rejoin the sentence
    splitted_tokens = tokenizer.batch_encode_plus(words,return_attention_mask=False,return_token_type_ids=False,add_special_tokens=False)['input_ids']
    masked_sentence=' '.join([tokenizer.decode([ mask_id if np.random.rand() < mask_probability else tok_id for tok_id in word]).replace(" ",'') for word in splitted_tokens])
    masked_sentences.append(masked_sentence)



# masked_sentence = ' '.join([mask_token if np.random.rand() < mask_probability else word for word in words])
        

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/cgrdj/nltk_data'
    - '/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/nltk_data'
    - '/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/share/nltk_data'
    - '/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [369]:
masked_sentence

[6904, 5104, 2546, 1012, 6904, 5104]

In [416]:
remove_spaces_table=str.maketrans('', '', ' ')


In [431]:
%%timeit 
tokenizer.decode([ mask_id if np.random.rand() < mask_probability else tok_id for tok_id in splitted_tokens[12]]).replace(" ",'')


53.5 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [429]:
%%timeit -n 100000
tokenizer.decode([ mask_id if np.random.rand() < mask_probability else tok_id for tok_id in splitted_tokens[12]]).translate(remove_spaces_table)


59.8 µs ± 4.79 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [385]:
tokenizer('f')

{'input_ids': [101, 1042, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [227]:
tokenizer.convert_tokens_to_string(tokenizer(sentence)['input_ids'])



TypeError: argument 'tokens': 'int' object cannot be converted to 'PyString'

In [212]:
mask_probability=0.15
mask_token = tokenizer.mask_token  # Get the mask token
tokens = tokenizer.tokenize(sentence)
# Decide randomly which tokens to mask
masked_indices = np.random.rand(len(tokens)) < mask_probability
# Replace selected tokens with the mask token
masked_tokens = [mask_token if mask else token for token, mask in zip(tokens, masked_indices)]
# Convert the list of tokens back to a string
masked_sentence = tokenizer.convert_tokens_to_string(masked_tokens)
# Add the masked sentence to the list
# masked_sentences.append(masked_sentence)
masked_sentence

"asdas [MASK] adsf sda fads ' asd gas [MASK]g aasdfasdf / dsaf sad [MASK]d, [MASK]fadsf. fadsf. fa [MASK], fadsfa"

In [208]:
masked_sentence

"as [MASK]df adsf sd [MASK] fads ' asd gas [MASK]g aa [MASK]fasdf [MASK] [MASK] [MASK] sadfsd, dasfadsf. [MASK] [MASK] [MASK]. fads [MASK] fa [MASK]fa"

In [203]:

word_tokenize(sentence)




['asdasdf',
 'adsf',
 'sda',
 'fads',
 "'asd",
 'gasdg',
 'aasdfasdf',
 '/dsaf',
 'sadfsd',
 ',',
 'dasfadsf',
 '.',
 'fadsf.fads',
 ',',
 'fadsfa']

# Data

In [192]:
dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))

Dataset downloaded here: /Users/cgrdj/Documents/Code/sentence-transformers/datasets/scifact


In [193]:
corpus, queries, qrels = GenericDataLoader(data_path).load(split="train") # or split = "train" or "dev"
unsupervised_train_data =  list(queries.values())#+[data['title']+' \n '+data['text'] for data in list(corpus.values())]
random.Random(0).shuffle(unsupervised_train_data)


2024-03-16 00:49:24 - Loading Corpus...


100%|██████████| 5183/5183 [00:00<00:00, 11485.08it/s]

2024-03-16 00:49:24 - Loaded 5183 TRAIN Documents.
2024-03-16 00:49:24 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 ver

In [194]:
data_path = "datasets/scifact"
test_corpus, test_queries, test_qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

2024-03-16 00:49:25 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

100%|██████████| 5183/5183 [00:00<00:00, 16216.28it/s]


2024-03-16 00:49:26 - Loaded 5183 TEST Documents.
2024-03-16 00:49:26 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers